In [1]:
# импорты
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from eli5.lime import TextExplainer

import regex as re
import pandas as pd
import numpy as np
import pickle

Using TensorFlow backend.


In [3]:
# модель
class KerasTextClassifier(BaseEstimator, TransformerMixin):
    '''Wrapper class for keras text classification models that takes raw text as input.'''
    
    def __init__(self, max_words=30000, input_length=1000, emb_dim=20, 
                 n_classes=4, epochs=5, batch_size=32, model_path="neural_model.hdf5", 
                tokenizer_path="tokenizer.pkl"):
        self.max_words = max_words
        self.input_length = input_length
        self.emb_dim = emb_dim
        self.n_classes = n_classes
        self.epochs = epochs
        self.bs = batch_size
        self.model_path = model_path
        self.model = self._get_model()
        self.tokenizer_path = tokenizer_path
        self.tokenizer = TfidfVectorizer(token_pattern='[a-zA-zА-яа-яёЁ]+', 
                                         max_features=self.input_length, 
                                        ngram_range=(1,3))
    
    def _get_model(self):
        model = Sequential()
        model.add(Dense(256, input_dim=self.input_length, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(self.n_classes, activation='softmax'))
        opt = Adam(learning_rate=0.01)
        model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
        return model
    
    def _get_sequences(self, texts):
        return self.tokenizer.transform(texts).toarray()
    
    def _preprocess(self, texts):
        return [re.sub(r"\d", "DIGIT", x) for x in texts]
    
    def fit(self, X, y):
        '''
        Fit the vocabulary and the model.
        
        :params:
        X: list of texts.
        y: labels.
        '''
        
        model_checkpoint_callback = ModelCheckpoint(
            filepath=self.model_path,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True)
        
        self.tokenizer.fit(self._preprocess(X))
        with open(self.tokenizer_path, 'wb') as handle:
            pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        seqs = self._get_sequences(self._preprocess(X))
        self.model.fit(seqs, y, batch_size=self.bs, epochs=self.epochs, validation_split=0.1, 
                      callbacks=[model_checkpoint_callback])
    
    def predict_proba(self, X, y=None):
        seqs = self._get_sequences(self._preprocess(X))
        return self.model.predict(seqs)
    
    def predict(self, X, y=None):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return accuracy_score(y, y_pred)
    
    def save(self):
        with open(self.tokenizer_path, 'wb') as handle:
            pickle.dump(self.tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        self.model.save(self.model_path)
        
    def load(self):
        with open(self.tokenizer_path, 'rb') as handle:
            self.tokenizer = pickle.load(handle)
        self.model = load_model(self.model_path)
        
# top_k accuracy
def top_k_accuracy(y_pred, y_true, k=3):
    top_preds = np.argsort(y_pred, axis=1)[:, -k:]
    return np.mean([int(y_true[n]) in pred for n, pred in enumerate(top_preds)])

In [4]:
# обезличенные данные резюме, полученные с hh
data = pd.read_csv("parsed_data.csv").append(pd.read_csv("parsed_data_ds.csv"), sort=False)

In [5]:
# для двух вакансий не было достаточно данных на hh, так что исключим их 
# для качества классификации и баланса классов
data = data.groupby("name").filter(lambda x: len(x) == 60)

In [6]:
names = {name: n for n, name in enumerate(data["name"].unique())}
reverse_names = {n: name for n, name in enumerate(data["name"].unique())}

In [7]:
data["name"] = data["name"].map(names)

In [8]:
X_train, X_test = train_test_split(data, stratify=data["name"].values, random_state=42)

In [9]:
# классы - вакансии с сайта https://www.gpbspace.ru/vacancy/
print(names)

{'Go разработчик': 0, 'Главный инженер по сопровождению': 1, 'Эксперт направления моделирования резервов': 2, 'Data Engineer': 3, 'Аналитик SAS': 4, 'Аналитик банковских рисков': 5, 'Главный инженер по тестированию (автоматизация)': 6, 'Ведущий DevOps инженер': 7, 'Дежурный инженер сопровождения банковских систем': 8, 'Дизайнер мобильных интерфейсов': 9, 'Разработчик Front-end (Middle)': 10, 'Системный аналитик DWH': 11, 'Аналитик системы принятия решений': 12, 'Инженер DevOps': 13, 'Главный разработчик Back-end Java': 14, 'Разработчик RPA': 15, 'Разработчик Front-end (REACT)': 16, 'Системный аналитик': 17, 'Архитектор': 18, 'Системный аналитик (проекты розничного блока)': 19, 'Системный аналитик (базы данных)': 20, 'Аналитик (web приложения)': 21, 'Бизнес-технолог': 22, 'Frontend разработчик': 23, 'Руководитель разработки JAVA': 24, 'Senior Data Scientist': 25}


In [20]:
text_model = KerasTextClassifier(epochs=50, n_classes=data["name"].nunique(), input_length=1000)

In [21]:
text_model.fit(X_train.resume_text, to_categorical(X_train.name))

Epoch 1/50
33/33 [==============================] - 0s 10ms/step - loss: 3.1915 - accuracy: 0.1899 - val_loss: 3.0917 - val_accuracy: 0.2821
Epoch 2/50
33/33 [==============================] - 0s 5ms/step - loss: 2.9005 - accuracy: 0.4321 - val_loss: 2.7706 - val_accuracy: 0.3248
Epoch 3/50
33/33 [==============================] - 0s 5ms/step - loss: 2.4891 - accuracy: 0.4824 - val_loss: 2.3876 - val_accuracy: 0.3675
Epoch 4/50
33/33 [==============================] - 0s 6ms/step - loss: 2.0856 - accuracy: 0.5147 - val_loss: 2.1024 - val_accuracy: 0.3761
Epoch 5/50
33/33 [==============================] - 0s 4ms/step - loss: 1.7843 - accuracy: 0.5537 - val_loss: 1.9224 - val_accuracy: 0.3504
Epoch 6/50
33/33 [==============================] - 0s 5ms/step - loss: 1.5816 - accuracy: 0.5764 - val_loss: 1.8077 - val_accuracy: 0.3761
Epoch 7/50
33/33 [==============================] - 0s 7ms/step - loss: 1.4372 - accuracy: 0.6087 - val_loss: 1.7429 - val_accuracy: 0.3932
Epoch 8/50
33/33 [=

In [22]:
# топ-3 точность
100 * top_k_accuracy(text_model.predict_proba(X_test.resume_text), X_test.name.values, 3)

79.23076923076923

In [23]:
# топ-5 точность
100 * top_k_accuracy(text_model.predict_proba(X_test.resume_text), X_test.name.values, 5)

90.25641025641026

In [24]:
# сохраняем
text_model.save()

In [31]:
text = """Ерохин Артем Игоревич

Мужчина, 28 лет, родился 6 февраля 1992

+7 (925) 5299117
ggofat@gmail.com — предпочитаемый способ связи
Skype: ggofat

Проживает: Москва, м. Котельники
Гражданство: Россия, есть разрешение на работу: Россия
Не готов к переезду, готов к редким командировкам

Желаемая должность и зарплата
Data Scientist
Информационные технологии, интернет, телеком

• Программирование, Разработка
• Аналитик

Занятость: полная занятость
График работы: гибкий график, полный день
Желательное время в пути до работы: не более полутора часов

Опыт работы — 5 лет 10 месяцев
Декабрь 2018 —
настоящее время
2 года

YouDo
Москва, www.youdo.ru
Data Scientist
Основные направления работы:
1. Рекомендательные системы;
2. Предсказание популярности задания, моделирование цены;
3. Кластеризация текстовых данных, выделение намерений, именованных сущностей;
4. Классификация заданий по существующей категоризации. Доработка категоризации;
5. Автоматизированное противодействие мошенничеству;
6. Автоматизация А/Б тестирования;
7. Прогнозирование оттока;
8. Осуществление процесса разметки данных.

Проекты:
1. Рекомендации заданий исполнителям;
2. Моделирование цены задания: непосредственное предсказание цены и моделирование
посредством предсказания спроса и предложения;
3. Уточнение и дополнение существующей категоризации на основании данных сервиса;
4. Классификатор мошеннических заданий;
5. Классификатор категории задания;
6. Выделение основного интентов запросов пользователей;
7. Прогнозирование оттока исполнителей;
8. Разметка данных в Яндекс.Толока.

Январь 2018 —
Декабрь 2018
1 год

Стек: Sklearn, LightGBM, Turicreate, Implicit, Spacy, DeepPavlov, Natasha, Яндекс.Толока
Fasten Inc. (ГК Везет)
Data Scientist

Резюме обновлено 17 апреля 2020 в 10:47

Основные направления работы:
1. Работа с географическими данными дорожной сети и навигационными отметками GPS;
2. Работа над задачами трассировки поездки и оценки загруженности дорожной сети;
3. Реализация функционала на Python;
4. Обучение ML-моделей, а так же оценка результатов их работы
5. Консультирование коллег по возникающим вопросам, связанным с анализом данных

Проект: Разработка моделей оценки характеристик пользовательской поездки:
- оценка скорости ребер графа дорожной сети из данных поездок (очистка данных, выбор способа
оценки, получение алгоритма)
- "выравнивание" полученных простым алгоритмом оценок характеристик посредством машинного
обучения (создание скриптом для автоматического обучения модели, "упаковка" модели в контейнер)

Стек: Sklearn, NetworkX, Keras, Tensorflow Serving, Docker
МосгортрансНИИпроект, ГУП
Москва
Начальник отдела анализа данных
- Налаживание процесса очистки, трансформации и хранения данных
- Обработка данных с помощью кластера Spark
- Обработка и визуализация данных с помощью pandas, sklearn, seaborn, folium, basemap
- Работа с AWS
- Взаимодействие, общение с другими государственными организациями и держателями данных
- Руководство двумя группами аналитиков (анализ данных и транспортное моделирование)
- Планирование работы отдела

Проекты:
1. Матрица корреспонденций НГПТ (наземного городского пассажирского транспорта);
2. Решение задачи map-matching для получения данных маршрутов;
3. Кластеризация пассажиров НГПТ;
4. Разработка базы данных управления;
5. Разработка отчетности о качестве обслуживания НГПТ.

Стек: NetworkX, Geopandas, Shapely, Pandas, Sklearn, PosrgresSQL (+PostGIS), Spark.
МосгортрансНИИпроект, ГУП
Москва
Старший аналитик
- Разработка модели транспортного спроса
- Написание скриптов для обработки и очистки данных, визуализация данных
- Работа с AWS
- Руководство двумя аналитиками
Объединенная компания Афиши и Рамблера
Москва, rambler.ru/
Аналитик
- Выполнение задач аналитики проектов Rambler&Co, маркетинга и редакции: настройка систем
аналитики, обработка полученных данных и интерпретация результатов;
- Постановка задач разработчикам на внесение изменений в код для выполнения задач аналитики,
тестирование и выявление ошибок в коде;
- Обработка и интерпретация данных различных систем (GoogleAnalytics, Яндекс метрика, ATInternet,
Adfox, TNS, Distimo и др);
- Работа с API вышеперечисленных систем (GA, Яндекс метрика, Flurry, Adfox, Google webmaster tools);

Февраль 2017 —
Январь 2018
1 год

Май 2016 — Январь
2017
9 месяцев

Апрель 2015 — Май
2016
1 год 2 месяца

Ерохин Артем  •  Резюме обновлено 17 апреля 2020 в 10:47

- Составление скриптов для выгрузки и визуализации данных (PHP 5.4, Python 2.7);
- Статистический анализ результатов А/Б тестирования продуктов;
- Визуализация данных в системе Tableau.

Октябрь 2014 —
Ноябрь 2014
2 месяца

КонсультантПлюс
Москва, www.consultant.ru/wanted/vacancy/
Стажер
Работа над междисциплинарными проектами со стажерами-юристами

Образование
Высшее

2015

МАИ
Прикладной математики и физики

Электронные сертификаты
2017

Специализация: Машинное обучение и анализ данных

Ключевые навыки
Знание языков

Русский — Родной
Английский — B2 — Средне-продвинутый

Навыки

 Python      SQL      Статистический анализ      Математическая статистика      MySQL 
 Аналитические исследования      Git      Анализ данных      Spark      PostgreSQL 
 Статистика      AWS 

Дополнительная информация
Обо мне

Хобби:
- Литература, спортивная версия "Что?Где?Когда?"

Дополнительно:
Профиль Kaggle - https://www.kaggle.com/ggofat
Github - https://github.com/artyerokhin
Спикер конференции DataStart2019 - https://datastart.ru/msk-autumn-2019/"""

In [45]:
names

{'Go разработчик': 0,
 'Главный инженер по сопровождению': 1,
 'Эксперт направления моделирования резервов': 2,
 'Data Engineer': 3,
 'Аналитик SAS': 4,
 'Аналитик банковских рисков': 5,
 'Главный инженер по тестированию (автоматизация)': 6,
 'Ведущий DevOps инженер': 7,
 'Дежурный инженер сопровождения банковских систем': 8,
 'Дизайнер мобильных интерфейсов': 9,
 'Разработчик Front-end (Middle)': 10,
 'Системный аналитик DWH': 11,
 'Аналитик системы принятия решений': 12,
 'Инженер DevOps': 13,
 'Главный разработчик Back-end Java': 14,
 'Разработчик RPA': 15,
 'Разработчик Front-end (REACT)': 16,
 'Системный аналитик': 17,
 'Архитектор': 18,
 'Системный аналитик (проекты розничного блока)': 19,
 'Системный аналитик (базы данных)': 20,
 'Аналитик (web приложения)': 21,
 'Бизнес-технолог': 22,
 'Frontend разработчик': 23,
 'Руководитель разработки JAVA': 24,
 'Senior Data Scientist': 25}

In [46]:
reverse_names

{0: 'Go разработчик',
 1: 'Главный инженер по сопровождению',
 2: 'Эксперт направления моделирования резервов',
 3: 'Data Engineer',
 4: 'Аналитик SAS',
 5: 'Аналитик банковских рисков',
 6: 'Главный инженер по тестированию (автоматизация)',
 7: 'Ведущий DevOps инженер',
 8: 'Дежурный инженер сопровождения банковских систем',
 9: 'Дизайнер мобильных интерфейсов',
 10: 'Разработчик Front-end (Middle)',
 11: 'Системный аналитик DWH',
 12: 'Аналитик системы принятия решений',
 13: 'Инженер DevOps',
 14: 'Главный разработчик Back-end Java',
 15: 'Разработчик RPA',
 16: 'Разработчик Front-end (REACT)',
 17: 'Системный аналитик',
 18: 'Архитектор',
 19: 'Системный аналитик (проекты розничного блока)',
 20: 'Системный аналитик (базы данных)',
 21: 'Аналитик (web приложения)',
 22: 'Бизнес-технолог',
 23: 'Frontend разработчик',
 24: 'Руководитель разработки JAVA',
 25: 'Senior Data Scientist'}

In [47]:
CLASSES = {
    0: "Go разработчик",
    1: "Главный инженер по сопровождению",
    2: "Эксперт направления моделирования резервов",
    3: "Data Engineer",
    4: "Аналитик SAS",
    5: "Аналитик банковских рисков",
    6: "Главный инженер по тестированию (автоматизация)",
    7: "Ведущий DevOps инженер",
    8: "Дежурный инженер сопровождения банковских систем",
    9: "Дизайнер мобильных интерфейсов",
    10: "Разработчик Front-end (Middle)",
    11: "Системный аналитик DWH",
    12: "Аналитик системы принятия решений",
    13: "Инженер DevOps",
    14: "Главный разработчик Back-end Java",
    15: "Разработчик RPA",
    16: "Разработчик Front-end (REACT)",
    17: "Системный аналитик",
    18: "Архитектор",
    19: "Системный аналитик (проекты розничного блока)",
    20: "Системный аналитик (базы данных)",
    21: "Аналитик (web приложения)",
    22: "Бизнес-технолог",
    23: "Frontend разработчик",
    24: "Руководитель разработки JAVA",
    25: "Senior Data Scientist",
}